<a href="https://colab.research.google.com/github/thesamasaurus/kaldheim-ml-bot/blob/main/format_all.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Format all drafts into string and onehot representations


### Importing

In [ ]:
import os
import re
import pandas as pd
import copy
import numpy as np
import pickle
import re
from google.colab import drive
import time

### Gathering Files

In [ ]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
g = open("/content/gdrive/MyDrive/Colab Notebooks/17_lands_scrapes/kaldheim/master_hash", "rb")
master_hash = pickle.load(g)

In [ ]:
master_hash["name_to_url"]["Frost Bite"]

['https://c1.scryfall.com/file/scryfall-cards/border_crop/front/9/4/9423318a-c5a8-48d2-92f5-280d15a050a6.jpg?1610996863',
 'https://c1.scryfall.com/file/scryfall-cards/border_crop/front/9/4/9423318a-c5a8-48d2-92f5-280d15a050a6.jpg?1614987802']

In [ ]:
h = open("/content/gdrive/MyDrive/Colab Notebooks/17_lands_scrapes/kaldheim/all_scrapes", "rb")
all_scrapes = pickle.load(h)
                     

In [ ]:
all_scrapes

,pick_1,pick_2,pick_3,pick_4,pick_5,pick_6,pick_7,pick_8,pick_9,pick_10,pick_11,pick_12,pick_13,pick_14,pick_15,pick_16,pick_17,pick_18,pick_19,pick_20,pick_21,pick_22,pick_23,pick_24,pick_25,pick_26,pick_27,pick_28,pick_29,pick_30,pick_31,pick_32,pick_33,pick_34,pick_35,pick_36,pick_37,pick_38,pick_39,pick_40,pick_41,pick_42,pick_43,pick_44
0,"<div class=""card_slot""><div class=""card_previe...","<div class=""card_slot""><div class=""card_previe...","<div class=""card_slot""><div class=""card_previe...","<div class=""card_slot""><div class=""card_previe...","<div class=""card_slot""><div class=""card_previe...","<div class=""card_slot""><div class=""card_previe...","<div class=""card_slot""><div class=""card_previe...","<div class=""card_slot""><div class=""card_previe...","<div class=""card_slot""><div class=""card_previe...","<div class=""card_slot""><div class=""card_previe...","<div class=""card_slot""><div class=""card_previe...","<div class=""card_slot""><div class=""card_previe...","<div class=""card_slot""><div class=""card_previe...","<div class=""card_slot""><div class=""card_previe...","<div class=""card_slot""><div class=""card_previe...","<div class=""card_slot""><div class=""card_previe...","<div class=""card_slot""><div class=""card_previe...","<div class=""card_slot""><div class=""card_previe...","<div class=""card_slot""><div class=""card_previe...","<div class=""card_slot""><div class=""card_previe...","<div class=""card_slot""><div class=""card_previe...","<div class=""card_slot""><div class=""card_previe...","<div class=""card_slot""><div class=""card_previe...","<div class=""card_slot""><div class=""card_previe...","<div class=""card_slot""><div class=""card_previe...","<div class=""card_slot""><div class=""card_previe...","<div class=""card_slot""><div class=""card_previe...","<div class=""card_slot""><div class=""card_previe...","<div class=""card_slot""><div class=""card_previe...","<div class=""card_slot""><div class=""card_previe...","<div class=""card_slot""><div class=""card_previe...","<div class=""card_slot""><div class=""card_previe...","<div class=""card_slot""><div class=""card_previe...","<div class=""card_slot""><div class=""card_previe...","<div class=""card_slot""><div class=""card_previe...","<div class=""card_slot""><div class=""card_previe...","<div class=""card_slot""><div class=""card_previe...","<div class=""card_slot""><div class=""card_previe...","<div class=""card_slot""><div class=""card_previe...","<div class=""card_slot""><div class=""card_previe...","<div class=""card_slot""><div class=""card_previe...","<div class=""card_slot""><div class=""card_previe...","<div class=""card_slot""><div class=""card_previe...","<div class=""card_slot""><div class=""card_previe..."
1,"<div class=""card_slot""><div class=""card_previe...","<div class=""card_slot""><div class=""card_previe...","<div class=""card_slot""><div class=""card_previe...","<div class=""card_slot""><div class=""card_previe...","<div class=""card_slot""><div class=""card_previe...","<div class=""card_slot""><div class=""card_previe...","<div class=""card_slot""><div class=""card_previe...","<div class=""card_slot""><div class=""card_previe...","<div class=""card_slot""><div class=""card_previe...","<div class=""card_slot""><div class=""card_previe...","<div class=""card_slot""><div class=""card_previe...","<div class=""card_slot""><div class=""card_previe...","<div class=""card_slot""><div class=""card_previe...","<div class=""card_slot""><div class=""card_previe...","<div class=""card_slot""><div class=""card_previe...","<div class=""card_slot""><div class=""card_previe...","<div class=""card_slot""><div class=""card_previe...","<div class=""card_slot""><div class=""card_previe...","<div class=""card_slot""><div class=""card_previe...","<div class=""card_slot""><div class=""card_previe...","<div class=""card_slot""><div class=""card_previe...","<div class=""card_slot""><div class=""card_previe...","<div class=""card_slo

### Dropping dead drafts missed in cleaning

In [ ]:
all_scrapes = all_scrapes.drop([184, 289, 612, 634, 867])
all_scrapes = all_scrapes.reset_index(drop=True)

### Making list of all picks

In [ ]:
def create_full_draft(dataframe, draft):
  draft_arr = []
  for i in range(len(dataframe.columns)):
    draft_arr.append(dataframe["pick_" + str(i+1)][draft])
  return draft_arr

In [ ]:
draft_one_full = create_full_draft(all_scrapes, 0)

### Defining regex to turn string into list of urls

In [ ]:
def take_regex_of_draft(draft_array):
  pack_array = []
  pick_array = []
  for i in range(len(draft_array)):
    # generating packs
    pack_clean = []
    # print("pick:", i)
    pack_dirty = re.findall("src=[a-zA-Z0-9~@#$^*()_+=[\]{}|\\,.?: \"\'/-]*", draft_array[i])
    for j in range(len(pack_dirty)):
      if (j % 2 == 0):
        pack_clean.append(pack_dirty[j][5:-1])

    #generating picks
    pick_dirty = re.findall("over pick\" src=[a-zA-Z0-9~@#$^*()_+=[\]{}|\\,.?: \"\'/-]*", draft_array[i])
    pick_clean = pick_dirty[0][16:-1]

    #appending output
    pack_array.append(pack_clean)
    pick_array.append(pick_clean)
  return pack_array, pick_array

In [ ]:
pack_array, pick_array = take_regex_of_draft(draft_one_full)

In [ ]:
pack_array[43]

['https://c1.scryfall.com/file/scryfall-cards/border_crop/front/e/4/e4802e6c-6921-4dac-bf44-7e09d6942b71.jpg?1610586564',
 'https://c1.scryfall.com/file/scryfall-cards/border_crop/front/3/f/3f23c487-ac4a-475b-ad5e-ec6f8678e668.jpg?1611054452']

### Creating pool array for each pick



In [ ]:
def create_pools(pick_array):
  pool_array = [[]]
  for i in range(len(pick_array)):
    current_pool = pool_array[i]
    new_pool = copy.deepcopy(current_pool)
    new_pool.append(pick_array[i])
    pool_array.append(new_pool)
  return pool_array

In [ ]:
pool_array = create_pools(pick_array)

In [ ]:
pool_array[2]

['https://c1.scryfall.com/file/scryfall-cards/border_crop/front/1/4/14dc88ee-bba9-4625-af0d-89f3762a0ead.jpg?1610586381',
 'https://c1.scryfall.com/file/scryfall-cards/border_crop/front/4/5/455ae615-20d7-4251-828d-72a3345d06f1.jpg?1610996855']

### Create master draft dict

In [ ]:
def create_master_draft_dict(num_of_drafts):
  master_dict = {}
  for i in range(num_of_drafts):
    master_dict["draft_" + str(i)] = {}
    for j in range(1, 45):
      master_dict["draft_" + str(i)]["pick_" + str(j)] = {}
  return master_dict

### Turning array into string



In [ ]:
def urls_to_strings_pack(array_of_urls, string_dict_name, master_draft_dict, draft_num):
  for i in range(0, 44):
    string_rep = []
    for url in array_of_urls[i]:
      if url in master_hash["url_to_name"].keys():
        string_rep.append(master_hash["url_to_name"][url])
      else:
        missing_urls[url] = ""
    master_draft_dict["draft_" + str(draft_num)]["pick_" + str(i+1)][string_dict_name] = string_rep
  return 

In [ ]:
missing_urls = {}

In [ ]:
def urls_to_strings_pick(array_of_urls, string_dict_name, master_draft_dict, draft_num):
  for pick in range(0, 44):
    string_rep = []
    if array_of_urls[pick] in master_hash["url_to_name"].keys():
      string_rep.append(master_hash["url_to_name"][array_of_urls[pick]])
    else:
      missing_urls[array_of_urls[pick]] = ""
    master_draft_dict["draft_" + str(draft_num)]["pick_" + str(pick+1)][string_dict_name] = string_rep
  return 

In [ ]:
def build_all_drafts(dataframe):
  master_draft_dict = create_master_draft_dict(len(all_scrapes["pick_1"]))
  for i in range(len(all_scrapes["pick_1"])):
    all_drafts = create_full_draft(all_scrapes, i)
    try:
      all_pack_array, all_pick_array = take_regex_of_draft(all_drafts)
    except:
      print("error, draft: ", i)
    all_pool_array = create_pools(all_pick_array)
    urls_to_strings_pack(all_pack_array, "pack", master_draft_dict, i)
    urls_to_strings_pack(all_pool_array, "pool", master_draft_dict, i)
    urls_to_strings_pick(all_pick_array, "pick", master_draft_dict, i)
  return master_draft_dict


### master_draft_dict is the dictionary that holds the string representation of every pick, pack, and pool for every draft in the dataset.  "missing urls" will be populated with any missing urls that are present in the drafts but not present in the master_hash.  If anything appears in the output of the cell below, your machine learing bot will not work.  You need to make sure every card url provided has a corresponding card name before you move forward.  Go into master hash and update the hash using an API request from scryfall.

In [ ]:
master_draft_dict = build_all_drafts(all_scrapes)
missing_urls

{}

### onehot encoding the master dict

In [ ]:
def find_onehot(string_or_array):
  if (isinstance(string_or_array, list)):
    onehot = np.zeros(len(master_hash["name_to_url"]), dtype=np.int8)
    for card_string in string_or_array:
      onehot[master_hash["name_to_index"][card_string]] += 1
  return onehot

In [ ]:
def create_onehot_dict(master_dict):
  draft_count = len(master_dict)
  pick_count = len(master_dict["draft_0"])
  types = ["pick", "pack", "pool"]
  master_draft_dict_onehot = create_master_draft_dict(draft_count)
  for i in range(0, draft_count):
    for j in range(1, pick_count):
      for type in types:
        onehot = find_onehot(master_dict["draft_" +str(i)]["pick_" + str(j)][type])
        master_draft_dict_onehot["draft_" +str(i)]["pick_" + str(j)][type] = onehot
  return master_draft_dict_onehot

### Alternate onehot encoding for reduced value early picks - this is currently not in use

In [ ]:
# def find_onehot_float(string_or_array):
#   if (isinstance(string_or_array, list)):
#     onehot = np.zeros(len(master_hash["name_to_url"]), dtype=np.float32)
#     for card_string in string_or_array:
#       onehot[master_hash["name_to_index"][card_string]] = 1
#   return onehot

In [ ]:
# def create_onehot_dict_augmented(master_dict, pool_intercept=0.66, pool_slope=0.03):
#   draft_count = len(master_dict)
#   pick_count = len(master_dict["draft_0"])
#   types = ["pick", "pack", "pool"]
#   master_draft_dict_onehot = create_master_draft_dict(draft_count)
#   for i in range(0, draft_count):
#     for j in range(1, pick_count):
#       for type in types:
#         if (type == "pool"):
#           onehot = find_onehot_float(master_dict["draft_" +str(i)]["pick_" + str(j)][type])
#           master_draft_dict_onehot["draft_" +str(i)]["pick_" + str(j)][type] = onehot* np.minimum(1, pool_slope*j + pool_intercept)
#         else:
#           onehot = find_onehot(master_dict["draft_" +str(i)]["pick_" + str(j)][type])
#           master_draft_dict_onehot["draft_" +str(i)]["pick_" + str(j)][type] = onehot
#   return master_draft_dict_onehot

### Creating a master dictionary that contains all pools, packs, and picks for every draft in the dataset

In [ ]:
master_draft_dict_onehot = create_onehot_dict(master_draft_dict)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

### Creating feature vector for each pool

In [ ]:
feature_vector_index = {}
feature_vector_index["Colorless"] = 0
feature_vector_index["W"] = 1
feature_vector_index["U"] = 2
feature_vector_index["B"] = 3
feature_vector_index["R"] = 4
feature_vector_index["G"] = 5
feature_vector_index[0.0] = 6
feature_vector_index[1.0] = 7
feature_vector_index[2.0] = 8
feature_vector_index[3.0] = 9
feature_vector_index[4.0] = 10
feature_vector_index[5.0] = 11
feature_vector_index[6.0] = 12
feature_vector_index[7.0] = 13

In [ ]:
def create_feature_vector(master_dict, master_dict_onehot):

  #how we will code the feature vector, "C" is colorless
  vector_reference = [ "Colorless", "W", "U", "B", "R", "G", 0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]

  for i in range(len(master_dict)): #for each draft
    for j in range(len(master_dict["draft_0"])): #for each pick
      pool_feature_vector = np.zeros(len(vector_reference), dtype=np.float32)  #create an empty feature vector
      for card_name in master_dict["draft_" + str(i)]["pick_" + str(j+1)]["pool"]:  #fetch the cards in pool for that pick
        
        # adding color to vector
        colors = master_hash["name_to_color"][card_name]  #fetch colors of each card, formatted ["W", "U"] for UW gold card
        if (len(colors) == 0):
          pool_feature_vector[feature_vector_index["Colorless"]] += 1  #if colorless, add one to colorless feature vector
        else:
          for color in colors: #else, for each color
            pool_feature_vector[feature_vector_index[color]] += (1/len(colors))  #add 1/number of colors in card.  0.5 W 0.5 U for UW
        
        # adding cmc to vector
        cmc = master_hash["name_to_cmc"][card_name]  #fetch cmc
        try:
          pool_feature_vector[feature_vector_index[cmc]] += 1  #see if cmc is 0-7
        except:
          pool_feature_vector[feature_vector_index[7.0]] += 1 #if not, its more than 7, so just add one to seven

      master_dict_onehot["draft_" + str(i)]["pick_" + str(j+1)]["feature_vector"] = pool_feature_vector  #save results in master_dictionary

  return 

### testing our results

In [ ]:
master_draft_dict_onehot = create_onehot_dict(master_draft_dict)

In [ ]:
create_feature_vector(master_draft_dict, master_draft_dict_onehot)

In [ ]:
master_draft_dict_onehot["draft_10"]["pick_5"]["feature_vector"]

array([0. , 1.5, 0. , 1. , 0. , 1.5, 0. , 0. , 1. , 0. , 3. , 0. , 0. ,
       0. ], dtype=float32)

## Define function to take onehot and spit out string of card

In [ ]:
def find_contense(one_hot):
    pack = []
    for i in range(0, len(one_hot)):
        if (one_hot[i] == 0):
            pass
        if (one_hot[i] > 0):
            pack.append(master_hash["index_to_name"][i])
    return pack

### Creating input arrays for ML

In [ ]:
def generate_input_arrays(onehot_dict=master_draft_dict_onehot):
  pack_list = []
  pool_list = []
  pick_list = []
  feature_vector_list = []
  draft_count = len(onehot_dict)
  pick_count = len(onehot_dict["draft_0"])
  types = ["pick", "pack", "pool", "feature_vector"]
  for i in range(0, draft_count):
    for j in range(1, pick_count):
      for type in types:
        if (type == "pool"):
          pool_list.append(onehot_dict["draft_" +str(i)]["pick_" + str(j)][type])
        if (type == "pack"):
          pack_list.append(onehot_dict["draft_" +str(i)]["pick_" + str(j)][type])
        if (type == "pick"):
          pick_list.append(onehot_dict["draft_" +str(i)]["pick_" + str(j)][type])
        if (type == "feature_vector"):
          feature_vector_list.append(onehot_dict["draft_" +str(i)]["pick_" + str(j)][type])
  return pack_list, pool_list, pick_list, feature_vector_list

In [ ]:
pack_list_testrun, pool_list_testrun, pick_list_testrun, feature_vector_list_testrun = generate_input_arrays(master_draft_dict_onehot)

### Saving files to main folder.  Please note:  Every time you add additional drafts to the dataset, you must rerun this notebook to overwrite the previous saves and append the new drafts to the save file.

In [ ]:
pack_list_testrun_save = open("/content/gdrive/MyDrive/Colab Notebooks/17_lands_scrapes/kaldheim/pack_list_testrun", "wb")
pickle.dump(pack_list_testrun, pack_list_testrun_save)
pack_list_testrun_save.close()

In [ ]:
pool_list_testrun_save = open("/content/gdrive/MyDrive/Colab Notebooks/17_lands_scrapes/kaldheim/pool_list_testrun", "wb")
pickle.dump(pool_list_testrun, pool_list_testrun_save)
pool_list_testrun_save.close()

In [ ]:
pick_list_testrun_save = open("/content/gdrive/MyDrive/Colab Notebooks/17_lands_scrapes/kaldheim/pick_list_testrun", "wb")
pickle.dump(pick_list_testrun, pick_list_testrun_save)
pick_list_testrun_save.close()

In [ ]:
feature_vector_list_testrun_save = open("/content/gdrive/MyDrive/Colab Notebooks/17_lands_scrapes/kaldheim/feature_vector_list_testrun", "wb")
pickle.dump(feature_vector_list_testrun, feature_vector_list_testrun_save)
feature_vector_list_testrun_save.close()